In [23]:
import re
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

import keras
from keras import losses, metrics, layers, ops

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [24]:
%env TF_GPU_ALLOCATOR=cuda_malloc_async

env: TF_GPU_ALLOCATOR=cuda_malloc_async


In [25]:
]]]]]]]]]]]]]]]]]]]]]
]]]]train_df = pd.read_csv('data/train.csv')

In [26]:
train_set_df = train_df[['agent1', 'agent2', 'EnglishRules', 'LudRules']]

In [27]:
lud_preprocessor = layers.TextVectorization(
    vocabulary='ludii_tokens.dic',
    standardize='strip_punctuation'
)

eng_preprocessor = layers.TextVectorization(
    vocabulary='eng_tokens.dic'
)

max_eng = 0
max_lud = 0

eng_uniques = train_set_df['EnglishRules'].unique()

# for engrul in tqdm(eng_uniques):
#     eng_vector = eng_preprocessor(engrul)
#     max_eng = max(max_eng, eng_vector.shape[0])

# print(max_eng)


In [28]:
lud_uniques = train_set_df['LudRules'].unique()

EQPMNT_RE = r'\(equipment'
RULESTRIP_RE = r'[^a-zA-Z\(\)\{\}]'

# for rule in lud_uniques:
#     start = re.search(EQPMNT_RE, rule).span()[0]
#     pure_rule = rule[start:]
#     pure_rule = re.sub(RULESTRIP_RE, ' ', pure_rule)
#     lud_vector = lud_preprocessor(pure_rule)
#     max_lud = max(max_lud, lud_vector.shape[0])

# print(max_lud)

In [29]:
agent1_uniques = train_set_df['agent1'].unique()
agent2_uniques = train_set_df['agent2'].unique()

set(agent1_uniques) >= set(agent2_uniques)

code_agent = {}
agent_code = {}

for id, agent in enumerate(agent1_uniques):
    code_agent[id] = agent
    agent_code[agent] = id


eng_vector.shape=(900,)

lud_vector.shape=(23700,)

agents_to_categorical.shape=(144)

In [30]:
# 900+23700+144 = 24744
# defining input shape and input data distribution

agents_len = 144
engvector_len = 900
ludvector_len = 23700

enter = layers.Input(shape=(24744,))
agent_input_layer = layers.Input(shape=(agents_len,))
agent_input_data = ops.slice(enter, (0,), (agents_len,))

engrul_input_data = ops.slice(enter, (agents_len,), (engvector_len,))

ludrul_input_data = ops.slice(enter, (agents_len+engvector_len,), (ludvector_len,))

In [31]:
# English rules LSTM features extractor.
# 
# Train set contains 384 preextracted features.
# Let`s assume that game rule contains these features and ather that describes 
# Number of these features is unknown and needs to be discovered
# presume that total amount of fetures, that are important for game result prediction is not less then double amount 
# of preextracted features and to be 800

FEATURES_NUM = 128
ENG_VECTOR_DIM = 128
LUD_VECTOR_DIM = 128


In [32]:
# EngRule LSTM

engrul_input_layer = layers.Input(shape=(1, engvector_len,))
flatten = layers.Flatten()(engrul_input_layer)
eng_emb = layers.Embedding(input_dim=3692, output_dim=ENG_VECTOR_DIM)(flatten)
eng_x = layers.LSTM(ENG_VECTOR_DIM)(eng_emb)
# eng_x = layers.LSTM(ENG_VECTOR_DIM)(eng_x)
eng_out = layers.Dense(FEATURES_NUM, activation='relu')(eng_x)

# model = keras.Model(inputs=[enter], outputs=[eng_out])

# model.summary()

In [11]:
# LudRule LSTM

ludrul_input_layer = layers.Input(shape=(1, ludvector_len,))
flatten = layers.Flatten()(ludrul_input_layer)
lud_emb = layers.Embedding(input_dim=1240, output_dim=LUD_VECTOR_DIM)(flatten)
lud_x = layers.LSTM(LUD_VECTOR_DIM)(lud_emb)
# lud_x = layers.LSTM(LUD_VECTOR_DIM)(lud_x)
lud_out = layers.Dense(FEATURES_NUM, activation='relu')(lud_x)


In [12]:
assemble = layers.Concatenate(axis=1)([agent_input_layer, eng_out, lud_out])
x = layers.Dense(1200, activation='relu')(assemble)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(1, activation='tanh')(x)

model = keras.Model(inputs=[agent_input_layer, engrul_input_layer, ludrul_input_layer],
                    outputs=[out])


In [13]:
# model.summary()

In [14]:
# keras.utils.plot_model(model,
#                        show_shapes=True,
#                        expand_nested=True,
#                        show_layer_activations=True)

## DataGenerator

In [15]:

class GameDataGenerator(keras.utils.Sequence):
    eqpmnt_re = r'\(equipment'
    rulestrip_re = r'[^a-zA-Z\(\)\{\}]'
    engvector_len = 900
    ludvector_len = 23700
    
    def __init__(self, list_IDs, dataframe, labels_dict, batch_size=32, shuffle=True):
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.labels_dict = labels_dict
        self.df = dataframe[['agent1', 'agent2', 'EnglishRules', 'LudRules']]
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.list_IDs))
        self._agent_code = agent_code
    
    @property
    def agent_code(self):
        return self._agent_code

    @agent_code.setter
    def agent_code(self):
        agents = self.df['agent1'].unique().tolist()
        agents.sort()
        ret = {}
        for id_, agent in enumerate(agents):
            ret[agent] = id_
        self._agent_code = ret

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, batch_num):
          # Generate indexes of the batch
        indexes = self.indexes[batch_num*self.batch_size:(batch_num+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_ends(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def _lud_prepare(self, rule) -> str:
        start = re.search(self.eqpmnt_re, rule).span()[0]
        pure_rule = rule[start:]
        pure_rule = re.sub(self.rulestrip_re, ' ', pure_rule)
        return pure_rule

    def __data_generation(self, list_IDs_temp):
        X_agents = []
        X_engvectors = []
        X_ludvectors = []
        y = []

        agents_num = len(self.agent_code)
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            row = self.df.loc[ID]
            agent1, agent2, engrul, ludrul = row

            y.append(self.labels_dict[ID])

            agent1_encoded = keras.utils.to_categorical(self.agent_code[agent1], agents_num)
            agent2_encoded = keras.utils.to_categorical(self.agent_code[agent2], agents_num)
            agents_encoded = np.hstack((agent1_encoded, agent2_encoded))
            X_agents.append(np.array(agents_encoded))

            engrul_vector = eng_preprocessor(engrul)
            engrul_vector = keras.utils.pad_sequences((engrul_vector,), maxlen=self.engvector_len)
            X_engvectors.append(engrul_vector)

            ludrul_vector = lud_preprocessor(self._lud_prepare(ludrul))
            ludrul_vector = keras.utils.pad_sequences((ludrul_vector,), maxlen=self.ludvector_len)
            X_ludvectors.append(ludrul_vector)

        return (np.array(X_agents), np.array(X_engvectors), np.array(X_ludvectors)), np.array(y)

In [16]:
train_shuffled_df = shuffle(train_df)

labels = train_shuffled_df['utility_agent1'].to_dict()

X_train, X_test = train_test_split(train_shuffled_df, test_size=0.25)

In [17]:
X_train.index

Index([103992,  94959,  29277, 212382, 171032, 110152, 175886,  27136, 185061,
        78839,
       ...
        27451,  24692, 133737, 165707,  57650,  10439, 103233, 146205,  57026,
       210045],
      dtype='int64', length=174925)

In [18]:
BATCH_SIZE = 2

In [19]:
train_generator = GameDataGenerator(X_train.index, train_shuffled_df, labels, batch_size=BATCH_SIZE)
test_generator = GameDataGenerator(X_test.index, train_shuffled_df, labels, batch_size=BATCH_SIZE)

In [20]:
model.compile(optimizer='adam',
              loss=losses.MeanSquaredError(),
              metrics=[metrics.MeanSquaredError()]
              )

In [21]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [22]:
model = keras.saving.load_model('lstm_both_epoch03_val_0.1407.keras')

In [23]:
keras.backend.clear_session()
gc.collect()

checkpoint = keras.callbacks.ModelCheckpoint(
    monitor='val_mean_squared_error',
    save_best_only=True,
    filepath='lstm_both_epoch{epoch:02d}_val_{val_mean_squared_error:0.4f}.keras'
)

history = model.fit(
    x=train_generator,
    epochs=10,
    validation_data=test_generator,
    verbose=1,
	callbacks=[checkpoint])

Epoch 1/10
 7678/87462 ━━━━━━━━━━━━━━━━━━━━ 11:42:43 528ms/step - loss: 0.1436 - mean_squared_error: 0.1436

2024-10-23 05:37:12.560367: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:310] gpu_async_0 cuMemAllocAsync failed to allocate 154537248 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 145883136/4086169600
2024-10-23 05:37:12.560408: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:315] Stats: Limit:                      2922512384
InUse:                       255196040
MaxInUse:                    409733284
NumAllocs:                   119006190
MaxAllocSize:                154537248
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2024-10-23 05:37:12.563687: E external/local_xla/xla/stream_executor/gpu/gpu_cudamallocasync_allocator.cc:64] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2024-10-23 05:37:12.563725: E external/local_xla/xla/stream_executor/g

InternalError: Graph execution error:

Detected at node gradient_tape/functional_1/lstm_1_1/CudnnRNNBackpropV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/dmytrot/miniconda3/envs/rapids-24.10/lib/python3.12/asyncio/base_events.py", line 1986, i